# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0674, -0.0030,  0.1457, -0.0500, -0.1743, -0.0353, -0.0180, -0.1405,
         -0.0563,  0.0619],
        [-0.1028,  0.0628,  0.0874, -0.1347, -0.2328,  0.0430, -0.0275, -0.1429,
          0.0402,  0.0187]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.2018, -0.1259,  0.0460,  0.1953,  0.0703,  0.0833,  0.0899,  0.4291,
         -0.1990, -0.1863],
        [ 0.2965, -0.0910,  0.1780,  0.1199,  0.0665,  0.0539,  0.1283,  0.4553,
         -0.1546, -0.1882]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1290,  0.3057,  0.1067,  0.0462,  0.0251, -0.0824, -0.0237, -0.0169,
         -0.1387,  0.0565],
        [-0.1503,  0.4056,  0.1067,  0.0500,  0.0244, -0.0489, -0.0729,  0.0005,
         -0.2841,  0.0917]], grad_fn=<AddmmBackward>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.1102, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1478, grad_fn=<SumBackward0>)